![](https://www.pata.org/wp-content/uploads/2014/09/TripAdvisor_Logo-300x119.png)
# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor
**По ходу задачи:**
* Прокачаем работу с pandas
* Научимся работать с Kaggle Notebooks
* Поймем как делать предобработку различных данных
* Научимся работать с пропущенными данными (Nan)
* Познакомимся с различными видами кодирования признаков
* Немного попробуем [Feature Engineering](https://ru.wikipedia.org/wiki/Конструирование_признаков) (генерировать новые признаки)
* И совсем немного затронем ML
* И многое другое...   



### И самое важное, все это вы сможете сделать самостоятельно!

*Этот Ноутбук являетсся Примером/Шаблоном к этому соревнованию (Baseline) и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.

> что такое baseline решение, зачем оно нужно и почему предоставлять baseline к соревнованию стало важным стандартом на kaggle и других площадках.   
**baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой, просто для примера. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline являеться хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

В контексте нашего соревнования baseline идет с небольшими примерами того, что можно делать с данными, и с инструкцией, что делать дальше, чтобы улучшить результат.  Вообще готовым решением это сложно назвать, так как используются всего 2 самых простых признака (а остальные исключаются).

# import

In [26]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime as dt
import json

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [27]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [28]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [29]:
DATA_DIR = '/kaggle/input/'
df_train = pd.read_csv(DATA_DIR+'sf-dst-restaurant-rating/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'sf-dst-restaurant-rating/kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'sf-dst-restaurant-rating/sample_submission.csv')


In [30]:
df_train.info()

In [31]:
df_train.head(5)

In [32]:
df_test.info()

In [33]:
df_test.head(5)

In [34]:
sample_submission.head(5)

In [35]:
sample_submission.info()

In [36]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [37]:
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [38]:
data.sample(5)

In [39]:
data.Reviews[1]

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

# Cleaning and Prepping Data
Обычно данные содержат в себе кучу мусора, который необходимо почистить, для того чтобы привести их в приемлемый формат. Чистка данных — это необходимый этап решения почти любой реальной задачи.   
![](https://analyticsindiamag.com/wp-content/uploads/2018/01/data-cleaning.png)

## 1. Обработка NAN 
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, **даже отсутствие информации может быть важным признаком!**   
По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак 

In [40]:
# Для примера я возьму столбец Number of Reviews
data['Number_of_Reviews_isNAN'] = pd.isna(data['Number of Reviews']).astype('uint8')

In [41]:
data['Number_of_Reviews_isNAN']

In [42]:
# Далее заполняем пропуски 0, вы можете попробовать заполнением средним или средним по городу и тд...
data['Number of Reviews'].fillna(0, inplace=True)

### 2. Обработка признаков
Для начала посмотрим какие признаки у нас могут быть категориальными.

In [43]:
data.nunique(dropna=False)

Какие признаки можно считать категориальными?

Для кодирования категориальных признаков есть множество подходов:
* Label Encoding
* One-Hot Encoding
* Target Encoding
* Hashing

Выбор кодирования зависит от признака и выбраной модели.
Не будем сейчас сильно погружаться в эту тематику, давайте посмотрим лучше пример с One-Hot Encoding:
![](https://i.imgur.com/mtimFxh.png)

In [44]:
# для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
data = pd.get_dummies(data, columns=[ 'City',], dummy_na=True)

In [45]:
data.head(5)

In [46]:
data.sample(5)

#### Возьмем следующий признак "Price Range".

In [47]:
data['Price Range'].value_counts()

По описанию 'Price Range' это - Цены в ресторане.  
Их можно поставить по возрастанию (значит это не категориальный признак). А это значит, что их можно заменить последовательными числами, например 1,2,3  
*Попробуйте сделать обработку этого признака уже самостоятельно!*

In [48]:
# Ваша обработка 'Price Range'

> Для некоторых алгоритмов МЛ даже для не категориальных признаков можно применить One-Hot Encoding, и это может улучшить качество модели. Пробуйте разные подходы к кодированию признака - никто не знает заранее, что может взлететь.

### Обработать другие признаки вы должны самостоятельно!
Для обработки других признаков вам возможно придется даже написать свою функцию, а может даже и не одну, но в этом и есть ваша практика в этом модуле!     
Следуя подсказкам в модуле вы сможете более подробно узнать, как сделать эти приобразования.

In [49]:
# тут ваш код на обработку других признаков
# .....

![](https://cs10.pikabu.ru/post_img/2018/09/06/11/1536261023140110012.jpg)

# EDA 
[Exploratory Data Analysis](https://ru.wikipedia.org/wiki/Разведочный_анализ_данных) - Анализ данных
На этом этапе мы строим графики, ищем закономерности, аномалии, выбросы или связи между признаками.
В общем цель этого этапа понять, что эти данные могут нам дать и как признаки могут быть взаимосвязаны между собой.
Понимание изначальных признаков позволит сгенерировать новые, более сильные и, тем самым, сделать нашу модель лучше.
![](https://miro.medium.com/max/2598/1*RXdMb7Uk6mGqWqPguHULaQ.png)

### Посмотрим распределение признака

In [50]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [51]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

А кто-то говорил, что французы любят поесть=) Посмотрим, как изменится распределение в большом городе:

In [52]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [53]:
# Нормируем ранк ресторана относительно максимального ранга в его городе
city_max_rank=df_train[['Ranking','City']].groupby(by='City').max()
city_max_rank.columns=['max_rank']
df_train=df_train.merge(city_max_rank, how='left', on='City')
df_train['Ranking']=df_train['Ranking']/df_train['max_rank']


In [54]:
# посмотрим на топ 10 городов

for x in (df_train['City'].value_counts())[0:10].index:
#     df_train['Ranking'][df_train['City'] == x].hist(bins=100)
    df_train['Number of Reviews'][(df_train['City'] == x) & df_train['Number of Reviews']<200 ].hist(bins=1000)
plt.show()

# sns.pairplot(df_train[['Ranking','Rating']])

In [55]:
df_train['Number of Reviews'][(df_train['Number of Reviews'] > 100) ].hist(bins=100)

In [56]:
df_train['Number of Reviews'].value_counts().sort_values(ascending=False)

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение.

>Подумайте как из этого можно сделать признак для вашей модели. Я покажу вам пример, как визуализация помогает находить взаимосвязи. А далее действуйте без подсказок =) 


### Посмотрим распределение целевой переменной

In [57]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

### Посмотрим распределение целевой переменной относительно признака

In [58]:
df_train['Ranking'][df_train['Rating'] == 4].hist(bins=100)

In [59]:
df_train['Ranking'][df_train['Rating'] < 5].hist(bins=100)

### И один из моих любимых - [корреляция признаков](https://ru.wikipedia.org/wiki/Корреляция)
На этом графике уже сейчас вы сможете заметить, как признаки связаны между собой и с целевой переменной.

In [60]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.drop(['sample'], axis=1).corr(),)

Вообще благодаря визуализации в этом датасете можно узнать много интересных фактов, например:
* где больше Пицерий в Мадриде или Лондоне?
* в каком городе кухня ресторанов более разнообразна?

придумайте свои вопрос и найдите на него ответ в данных)

# Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [19]:
# на всякий случай, заново подгружаем данные
DATA_DIR = '/kaggle/input/'
df_train = pd.read_csv(DATA_DIR+'sf-dst-restaurant-rating/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'sf-dst-restaurant-rating/kaggle_task.csv')

df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

# # загружаем список городов
df_cities = pd.read_csv(DATA_DIR+'cities-of-the-world/cities15000.csv', encoding='latin-1')
# , encoding='latin-1'
df_cities = df_cities[['asciiname','population']]

# загружаем список позитивных и негативных слов 
df_positive = pd.read_csv(DATA_DIR+'d/nickboyarsky/words-pos-neg/positive-words.txt', encoding='latin-1')
df_negative = pd.read_csv(DATA_DIR+'d/nickboyarsky/words-pos-neg/negative-words.txt', encoding='latin-1')



### Функции, которые я использовал для обработки данных

In [23]:
"""Функция для распарсивания отзывов. Ловим много ошибок в строках."""
# не использовал регулярные выражения, т.к. с ними провозился еще дольше, т.к. для вылавливания ошибок с ними было трудней
def parse_reviews_stage1(rev):
    ''' Получаем reviews в формате:
    review['reviews_txt'][1] - тексты обзоров
    review['reviews_dt'][1] - даты обзоров
    '''
    if  not pd.isna(rev): 
        # избавляемся от всех неправильных символов
        rev = str(rev).replace("\\\\",'-')

        rev = str(rev).replace("\\'",'-')
        rev = str(rev).replace("\\x7f",'-')
        rev = str(rev).replace("\\xa0",'-')
        rev = str(rev).replace("', nan]",'!@, !@ !@]')
        rev = str(rev).replace("[nan, '",'[!@ !@, !@')

        rev = str(rev).replace("['",'[!@')
        rev = str(rev).replace("[\"",'[!@')
        rev = str(rev).replace("', '",'!@, !@')
        rev = str(rev).replace("\", '",'!@, !@')

        rev = str(rev).replace("', \"",'!@, !@')
        rev = str(rev).replace("\", \"",'!@, !@')
        rev = str(rev).replace("']",'!@]')
        rev = str(rev).replace("\"]",'!@]')
        
        rev = str(rev).replace("'",'-')
        rev = str(rev).replace("\"",'-')
        
        rev = str(rev).replace("!@",'\"')
        
        rev = rev.replace('], [', '], "reviews_dt": [')
        rev = '{ "reviews_txt":' + rev + '}'
        rev = rev.replace('[[','[').replace(']]',']')

    return rev

def parse_reviews(rev):
    ''' Получаем reviews в формате:
    review['reviews_txt'][1] - тексты обзоров
    review['reviews_dt'][1] - даты обзоров
    '''
    if  not pd.isna(rev): 
        # парсим получившуюся строку в объекты словарь и список
        d = json.loads(parse_reviews_stage1(rev))
        
        # получаем дату отзыва.
        d['reviews_dt'] = [dt.datetime.strptime(date, '%m/%d/%Y').date() if len(date.split('/')[2])==4 else dt.datetime.strptime(date, '%m/%d/%y').date() for date in d['reviews_dt']]
        return d
    else:
        return {}

    
#подсчет плохих и хороших слов в отзывах
def rew_words_count(words_series, review_series):
    word_arr=list(words_series)
    word_cnt_arr=[]
    for s1 in review_series:
        word_count=0

        for w in word_arr:
            if w.lower() in s1.lower(): word_count += 1

        word_cnt_arr.append(word_count)

#     df_output[count_column_name]=word_cnt_arr
    return word_cnt_arr


In [ ]:
# df_output=data
# rew_array=[]
# for s1 in df_output['Reviews']:
#     try:
#         rew_array.append(parse_reviews(s1))
#     except:
#         print(s1, parse_reviews_stage1(s1)) # это для поиска строк, на которых парсер падает, и для добавления новых условий замены символов

# df_output['reviews_txt']=pd.DataFrame(rew_array, columns=['reviews_txt'], dtype='datetime64[ns]').astype(str)
# df_output['reviews_dt']=pd.DataFrame(rew_array, columns=['reviews_dt'], dtype='datetime64[ns]')

# # rew_words_count(df_positive['positive_words'],df_output['reviews_txt'],'good_word_cnt')
# # rew_words_count(df_negative['negative_words'],df_output['reviews_txt'],'bad_word_cnt')

# data=df_output

### Функция обработки данных

In [24]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    # убираем не нужные для модели признаки
    df_output.drop(['ID_TA'], axis = 1, inplace=True)
    
    
    # ################### 2. NAN ############################################################## 
    # Далее заполняем пропуски, вы можете попробовать заполнением средним или средним по городу и тд...
    data['Number_of_Reviews_isNAN'] = pd.isna(data['Number of Reviews']).astype('uint8')
    df_output['Number of Reviews'].fillna(0, inplace=True)
    # тут ваш код по обработке NAN
    # ....
    data['Cuisine Style_isNAN'] = pd.isna(data['Cuisine Style']).astype('uint8')
    df_output['Cuisine Style'].fillna(value="['Home kitchen123']", inplace=True)
    

 
    # ################### 3. Encoding ############################################################## 
    ##########
    """Нормируем ранк ресторана относительно максимального ранга в его городе"""
    city_max_rank=df_output[['Ranking','City']].groupby(by=['City']).max()
    city_max_rank.columns=['max_rank']
    df_output=df_output.merge(city_max_rank, how='left', on=['City'])
    df_output['Ranking2']=df_output['Ranking']/df_output['max_rank']

    
    """Попытался отнормировать также на рейтинге, но это нечестно :-).... хотя MAE получилась 0.038 :-)"""
#     city_max_rating_rank=df_output[['Ranking','City','Rating']].groupby(by=['City','Rating']).max()
#     city_max_rating_rank.columns=['max_rating_rank']
#     df_output=df_output.merge(city_max_rating_rank, how='left', on=['City','Rating'])
#     df_output['Ranking_rat']=df_output['Ranking']/df_output['max_rating_rank']
 
    df_output.drop(['Ranking'], axis = 1, inplace=True)
    ##########
    
    
    ##########
    """Подсчитываем количество видов кухонь, которые есть в ресторане"""
    cousine_cnt=[]
    for s1 in df_output['Cuisine Style']:
        cousine_cnt.append(len(json.loads(str(s1).replace("'",'"'))))

    df_output['Cuisine_count']=cousine_cnt   # Количество кухонь записываем в новый столбец
    ##########
    
    
    ##########
    """Указываем, является ресторан сетевым или отдельным"""
    rest_franshiza=df_output['Restaurant_id'].value_counts()[df_output['Restaurant_id'].value_counts()>1]
    df_output['set_rest']=df_output['Restaurant_id'].isin(rest_franshiza.index).apply(lambda x: 1 if x==True else 0)
  
    
    """Проставляем вместо диапазона цен числовые аналоги и заполняем пустые значения средним значением по городу внутри рейтинга"""
    data['Price Range_isNAN'] = pd.isna(data['Price Range']).astype('uint8')
    df_output['Price Range'] = df_output['Price Range'].replace(['$','$$ - $$$','$$$$'], [1,2,3]).astype(float)
    city_price_range=df_output[['Price Range','City']].groupby(by=['City']).mean()

    city_price_range.columns=['city_price_range']
    df_output=df_output.merge(city_price_range, how='left', on=['City'])
    df_output['Price Range'].fillna(df_output['city_price_range'], inplace=True) 

    df_output['Price Range2']=df_output['Price Range']
    # раскидываем получившиеся ценовые диапазоны, в том числе и средние, на новые столбцы
    df_output = pd.get_dummies(df_output, columns=['Price Range'], dummy_na=True)
    
   
    """для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na"""
    df_output['City2']=df_output['City']
    df_output = pd.get_dummies(df_output, columns=[ 'City2',], dummy_na=True)
    # тут ваш код не Encoding фитчей
    # ....

    # ################### 4. Feature Engineering ####################################################
    # тут ваш код не генерацию новых фитчей
    # ....
    """Добавляем население городов и нормируем им количество отзывов"""
    #добавляем в наш датасет население городов
    df_cities.drop_duplicates(['asciiname'], inplace=True)
    df_output=df_output.merge(df_cities, how='left', left_on=['City'], right_on=['asciiname'])

    # двух городов не оказалось в датасете
    df_cities2=pd.DataFrame(data={'asciiname2': ['Edinburgh', 'Oporto'], 'population2': [488000, 287591]})
    
    df_output=df_output.merge(df_cities2, how='left', left_on=['City'], right_on=['asciiname2'])
    df_output['population'].fillna(value=df_output['population2'], inplace=True)
    df_output.drop(['asciiname', 'asciiname2','population2'], axis = 1, inplace=True)
    df_output['population']=df_output['population'].astype(int)
    df_output['Number of Reviews2']=df_output['Number of Reviews']/df_output['population']
    
  
    """Парсим отзывы"""
    # парсим строки отзывов в списки с словари и записываем их в столбцы нашего датафрейма    
    data['Reviews_isNAN'] = df_output['Reviews'].apply(lambda x: 1 if x=='[[], []]' else 0)
    
    rew_array=[]
    for s1 in df_output['Reviews']:
        try:
            rew_array.append(parse_reviews(s1))
        except:
            print(s1, parse_reviews_stage1(s1)) # это для поиска строк, на которых парсер падает, и для добавления новых условий замены символов

    df_output['reviews_txt']=pd.DataFrame(rew_array, columns=['reviews_txt'], dtype='datetime64[ns]').astype(str)
    df_output['reviews_dt']=pd.DataFrame(rew_array, columns=['reviews_dt'], dtype='datetime64[ns]')
    
    
    # вычисляем количества плохих и хороших слов (долго выполняется)
    df_output['good_word_cnt']=rew_words_count(df_positive['positive_words'],df_output['reviews_txt'])
    df_output['bad_word_cnt']=rew_words_count(df_negative['negative_words'],df_output['reviews_txt'])

    # вычисляем дату последнего отзыва и количество дней между отзывами     
    df_output['lastReviewDate'] = df_output['Reviews'].apply(lambda x: parse_reviews(x)['reviews_dt'] if not pd.isna(x) else {})
    df_output['day_between_rew'] = df_output['lastReviewDate'].apply(lambda x: abs(x[1]-x[0]).total_seconds()//86400 if len(x)>1 else 100000)
    df_output['lastReviewDate'] = df_output['lastReviewDate'].apply(lambda x: sorted(x,reverse=True)[0] if len(x)!=0 else pd.NaT)
    df_output['lastReviewDate'] = df_output['lastReviewDate'].fillna(dt.date(1950,1,1))
    df_output['lastReviewDate'] = df_output['lastReviewDate'].apply(lambda x: (dt.datetime.now().date()-x).total_seconds()//86400)

    
    
    # ################### 5. Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
    df_output.drop(object_columns, axis = 1, inplace=True)
    
    return df_output


>По хорошему, можно было бы перевести эту большую функцию в класс и разбить на подфункции (согласно ООП). 

#### Запускаем и проверяем что получилось

In [25]:
df_preproc = preproc_data(data)
df_preproc.sample(10)
df_preproc.info()


In [62]:
df_preproc.info()
df_preproc.sample(10)


In [63]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)


**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [64]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [65]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [66]:
y_test

In [67]:

# X_test.drop(['max_rating_rank','Ranking_rat',], axis = 1, inplace=True)
# X_test.info()

# Model 
Сам ML

In [68]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [69]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [70]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [71]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [72]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(25).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [73]:
test_data.sample(10)

In [74]:
test_data = test_data.drop(['Rating'], axis=1)

In [75]:
sample_submission

In [81]:
predict_submission = model.predict(test_data)
sample_submission = pd.read_csv(DATA_DIR+'sf-dst-restaurant-rating/sample_submission.csv')


In [82]:
predict_submission

In [83]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

# What's next?
Или что делать, чтоб улучшить результат:
* Обработать оставшиеся признаки в понятный для машины формат
* Посмотреть, что еще можно извлечь из признаков
* Сгенерировать новые признаки
* Подгрузить дополнительные данные, например: по населению или благосостоянию городов
* Подобрать состав признаков

В общем, процесс творческий и весьма увлекательный! Удачи в соревновании!


In [80]:
dt.datetime.fromtimestamp(int(os.path.getmtime('submission.csv')))